In [1]:
import tensorflow as tf
from Preprocess import label_for_nn
from cnn_model import model

In [2]:
benign_path = '/home/user01/Desktop/DDSM/4_benign_Pleo_10_ROI_subpic'#input('良性子图路径')#
cancer_path = '/home/user01/Desktop/DDSM/4_cancer_Pleo_10_ROI_subpic'#input('恶性子图路径')#
saver_path = '/home/user01/Desktop/DDSM/model/AlexNet_v1'#input('模型保存路径')#
size = 128
tf.reset_default_graph()

In [3]:
def get_trainsets_and_testsets(benign_path,cancer_path):
    (train_x ,train_y),(test_x,test_y) = label_for_nn(benign_path,cancer_path,scale=0.2,kind='RGB')
    train_x = train_x.reshape(train_x.shape[0],size,size,3).astype('float32')/255.0
    test_x = test_x.reshape(test_x.shape[0],size,size,3).astype('float32')/255.0
    return  test_x ,test_y

In [4]:
def conv2d_P(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='SAME')

In [5]:
def conv2d(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='VALID')

In [6]:
def max_pooling2x2(x):
    return tf.nn.max_pool(x,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')

In [7]:
def cnn_test():
    x = tf.placeholder(tf.float32, [None, 128, 128, 3])
    y = tf.placeholder(tf.float32, [None, 2])
    keep_prob = tf.placeholder(tf.float32)
    y_pre = model(x,keep_prob)
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pre), reduction_indices=[1]))
    y_pre_label = tf.argmax(y_pre,1)
    y_label = tf.argmax(y,1)
    a = tf.add(y_pre_label, 1)
    b = tf.add(y_label, 3)
    c = tf.multiply(a, b)
    tp = tf.reduce_sum(tf.cast(tf.equal(c, 8), tf.float32))
    fp = tf.reduce_sum(tf.cast(tf.equal(c, 6), tf.float32))
    tn = tf.reduce_sum(tf.cast(tf.equal(c, 3), tf.float32))
    fn = tf.reduce_sum(tf.cast(tf.equal(c, 4), tf.float32))
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    tf.summary.scalar('test_Sensitivity', tpr)
    tf.summary.scalar('test_Specificity', tnr)
    correct_l1 = tf.equal(y_pre_label, y_label)
    accuracy = tf.reduce_mean(tf.cast(correct_l1, tf.float32))
    tf.summary.scalar('test_loss', cross_entropy)
    tf.summary.scalar('test_accuracy', accuracy)
    merged_summary_op = tf.summary.merge_all()
    saver = tf.train.Saver()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        test_accuracy = 0
        writer = tf.summary.FileWriter(saver_path, sess.graph)
        saver.restore(sess,saver_path+'/AlexNet')
        test_accuracy = 0
        test_sensitivity = 0
        test_specificity = 0
        test_loss = 0
        for n in range(0,100):
            test_x, test_y = get_trainsets_and_testsets(benign_path,cancer_path)
            acc,sen,spec,loss,summary = sess.run([accuracy,tpr,tnr,cross_entropy,merged_summary_op],feed_dict={x:test_x,y:test_y,keep_prob:1.0})
            test_accuracy += acc
            test_sensitivity += sen
            test_specificity +=spec
            test_loss += loss
            print("{0:3d} accuracy:{1:>10.9f} loss:{2:>10.9f} specificity:{3:>10.9f} sensitivity:{4:>10.9f}".format(n,acc,loss,sen,spec))
            writer.add_summary(summary=summary, global_step=n)
        print("accuracy:{0:<6.3f}% loss:{1:<6.3f}% specificity:{2:<6.3f}% sensitivity:{3:<6.3f}%".format(test_accuracy,test_loss,test_sensitivity,test_specificity))

In [8]:
cnn_test()

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
INFO:tensorflow:Restoring parameters from /home/user01/Desktop/DDSM/model/AlexNet_v1/AlexNet
  0 accuracy:0.959999979 loss:0.204924643 specificity:0.923076928 sensitivity:1.000000000
  1 accuracy:1.000000000 loss:0.008384398 specificity:1.000000000 sensitivity:1.000000000
  2 accuracy:1.000000000 loss:0.001599544 specificity:1.000000000 sensitivity:1.000000000
  3 accuracy:1.000000000 loss:0.010582102 specificity:1.000000000 sensitivity:1.000000000
  4 accuracy:0.983333349 loss:0.048856378 specificity:0.961538434 sensitivity:1.000000000
  5 accuracy:1.000000000 loss:0.000150355 specificity:1.000000000 sensitivity:1.000000000
  6 accuracy:1.000000000 loss:0.005218505 specificity:1.000000000 sensitivity:1.000000000
  7 accuracy:1.000000000 loss:0.006811084 specificity:1.000000000 sensitivity:1.000000000
  8 accuracy:0.981818199 loss:0.038483623 specificity:0.967741907 sensitivity:1.000000000

 89 accuracy:0.975609779 loss:0.041372962 specificity:0.944444418 sensitivity:1.000000000
 90 accuracy:0.962264180 loss:0.058346689 specificity:0.925925910 sensitivity:1.000000000
 91 accuracy:0.978260875 loss:0.047607303 specificity:0.954545438 sensitivity:1.000000000
 92 accuracy:0.980769217 loss:0.043971650 specificity:0.968750000 sensitivity:1.000000000
 93 accuracy:0.956521749 loss:0.220558643 specificity:0.920000017 sensitivity:1.000000000
 94 accuracy:0.977272749 loss:0.041627195 specificity:0.961538434 sensitivity:1.000000000
 95 accuracy:1.000000000 loss:0.000997724 specificity:1.000000000 sensitivity:1.000000000
 96 accuracy:0.974358976 loss:0.041619677 specificity:0.949999988 sensitivity:1.000000000
 97 accuracy:0.983870983 loss:0.028454915 specificity:0.966666639 sensitivity:1.000000000
 98 accuracy:0.981481493 loss:0.033702180 specificity:0.970588207 sensitivity:1.000000000
 99 accuracy:1.000000000 loss:0.004874317 specificity:1.000000000 sensitivity:1.000000000
accuracy:9